In [106]:
import numpy as np
import scipy as sc
import pandas as pd
from coin import *

N = 25
pHeads = 0.7
# H <-> 0, T <-> 1
coinDistr = Coin(pHeads=0.7)
def makeObservation(N=N, distr=coinDistr):
    return [coinDistr.flip() for i in range(N)]

In [107]:
bernoulliEntropy = lambda p : p * np.log2(1/p) + (1 - p) * np.log2(1/(1-p))
entropyX = bernoulliEntropy(pHeads)
print(entropyX)

0.8812908992306926


In [108]:
# binary coefficients denoting the number of strings x with k ones
coeffCol = np.array([sc.special.binom(N, k) for k in range(N+1)])

# probability that a particular string x has k ones
stringProbCol = np.array([((1 - pHeads) ** k) * (pHeads ** (N - k)) for k in range(N+1)])

# probability of having k 1's in a binary string of length N is C(N, k) * (1 - p)^{k} * (p)^{N - k}
probCol = coeffCol * stringProbCol
# 1/N * log(P(x))
logCol = (1/N) * np.log2(stringProbCol) 

table = np.array([coeffCol, probCol, logCol])
df = pd.DataFrame(table.T, columns=['C(N,k)', 'P(k)', '(1/N)log(P(x))'])
print(df)

       C(N,k)          P(k)  (1/N)log(P(x))
0         1.0  1.341069e-04       -0.514573
1        25.0  1.436859e-03       -0.563469
2       300.0  7.389562e-03       -0.612365
3      2300.0  2.427999e-02       -0.661260
4     12650.0  5.723140e-02       -0.710156
5     53130.0  1.030165e-01       -0.759052
6    177100.0  1.471665e-01       -0.807947
7    480700.0  1.711936e-01       -0.856843
8   1081575.0  1.650796e-01       -0.905739
9   2042975.0  1.336359e-01       -0.954634
10  3268760.0  9.163601e-02       -1.003530
11  4457400.0  5.355351e-02       -1.052426
12  5200300.0  2.677676e-02       -1.101322
13  5200300.0  1.147575e-02       -1.150217
14  4457400.0  4.215583e-03       -1.199113
15  3268760.0  1.324897e-03       -1.248009
16  2042975.0  3.548832e-04       -1.296904
17  1081575.0  8.051973e-05       -1.345800
18   480700.0  1.533709e-05       -1.394696
19   177100.0  2.421646e-06       -1.443591
20    53130.0  3.113545e-07       -1.492487
21    12650.0  3.177086e-08     

In [114]:
beta = 0.1
typicalSetCondition = np.absolute(-1*logCol - entropyX) < beta
typicalSetCardinality = np.sum(np.array([ coeffCol[i] if typicalSetCondition[i] else 0 for i in range(N)]))
prob = np.sum(np.array([coeffCol[k] * stringProbCol[k] if typicalSetCondition[k] else 0 for k in range(N)]))

print(typicalSetCardinality)
print(prob)

3782350.0
0.6170755344069891


In [115]:
delta = 0.1
smallestDeltaSufficentCardinality = 0
typicalSetIntersectionCardinality = 0
k = 0

while k <= N:
    currProb = np.sum(probCol[:k])
    if currProb >= 1 - delta:
        break
    numStrings = coeffCol[k]
    smallestDeltaSufficentCardinality += numStrings
    if typicalSetCondition[k]:
        typicalSetIntersectionCardinality += numStrings
        
    k += 1

print(smallestDeltaSufficentCardinality)
print(typicalSetIntersectionCardinality)

7119516.0
3782350.0
